In [1]:
import pandas as pd
import numpy as np

In [2]:
xnames = []
ynames = []
angle_names = []
for i in range(1, 10):
    for j in range(i + 1, 10):
        xnames.append("xd" + str(i) + str(j))
        ynames.append("yd" + str(i) + str(j))
        angle_names.append("phi1" + str(i) + str(j))
        angle_names.append("phi2" + str(i) + str(j))
        
triangle_names = []
for i in range(1, 10):
    for j in range(1, 10):
        if j == i:
            continue
        for k in range(j + 1, 10):
            if k == i:
                continue
            triangle_names.append("theta" + str(i) + str(j) + str(k))

In [3]:
colnames = []
colnames.extend(xnames)
colnames.extend(ynames)
colnames.extend(angle_names)
colnames.extend(triangle_names)

In [4]:
names = []
for i in range(1, 15):
    names.extend(colnames)
names.append("label")

In [5]:
for i in range(1, 15):
    for j in range(396*(i-1), 396*i):
        names[j] += "_frame" + str(i)

In [6]:
df1 = pd.read_csv("pose_features_train/train_features.csv", index_col=0, header=0, names=names) 
df2 = pd.read_csv("pose_features_train/train_features2.csv", index_col=0, header=0, names=names)
df3 = pd.read_csv("pose_features_train/train_features3.csv", index_col=0, header=0, names=names)
print(len(df1), len(df2), len(df3))

2761 1798 1211


In [7]:
total_df = [df1, df2, df3]
df = pd.concat(total_df)
len(df)

5770

In [8]:
df = df.drop_duplicates()
len(df)

5766

In [9]:
print("No Null Values:", not len(pd.isnull(df).any().values.nonzero()) == 0)
print("No Inf Values:", len(np.argwhere(np.isfinite(df.values) == 0)) == 0)

No Null Values: True
No Inf Values: True


In [10]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=400, random_state=0, max_depth = 15)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [38]:
# test_df = pd.read_csv("pose_features_test/test_features.csv", index_col=0, header=None)
# test_arr = test_df.iloc[:, :-1].values

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[471  43]
 [  8 632]]
              precision    recall  f1-score   support

         0.0       0.98      0.92      0.95       514
         1.0       0.94      0.99      0.96       640

    accuracy                           0.96      1154
   macro avg       0.96      0.95      0.95      1154
weighted avg       0.96      0.96      0.96      1154

0.9558058925476604


In [16]:
import pickle
filename = "./classifier.pkl"
with open(filename, 'wb') as file:
    pickle.dump(classifier, file)
# pickle.dump(model, open(filename, 'wb'))